In [6]:
import matplotlib.pyplot as plt

## Data structures

In [1]:
from __future__ import annotations
from dataclasses import dataclass
from functools import reduce
from typing import List, Tuple
from multimethod import multimethod
import math

# One Qbits class holding any number of qubits it's enough actually

@dataclass
class Qbits:
    vector : List[float] #column vector
    # tensor product: multiply each element of another array by
    # all the elements of this array
    @multimethod
    def __mul__(self, q: Qbits) -> Qbits:
        v = []
        for x in q.vector:
            for y in self.vector:
                v.append(x * y)
        return Qbits(v)
    def __len__(self) -> int:
        return len(self.vector)
    @multimethod
    def __mul__(self, s: float) -> Qbits:
        return Qbits([s * x for x in self.vector])
    @multimethod
    def __mul__(self, s: int) -> Qbits:
        return self.__mul__(float(s))
    @multimethod
    def __mul__(self, _):
        return NotImplemented
    @multimethod
    def __rmul__(self, s: float) -> Qbits:
        return self.__mul__(s)
    @multimethod
    def __rmul__(self, s: int) -> Qbits:
        return self.__rmul__(float(s))
    
@dataclass(frozen=True)
class Qbit:
    vector : List[float] #column vector
    @multimethod
    def __mul__(self, q: Qbit) -> Qbits:
        v = [0.] * 4
        v[0] = self.vector[0] * q.vector[0]
        v[1] = self.vector[1] * q.vector[0]
        v[2] = self.vector[0] * q.vector[1]
        v[3] = self.vector[1] * q.vector[1]
        return Qbits(v)
    @multimethod
    def __mul__(self, s: float) -> Qbit:
        return Qbit([s * self.vector[0], s * self.vector[1]])
    @multimethod
    def __mul__(self, s: int) -> Qbit:
        return self.__mul__(float(s))
    
    @multimethod
    def __rmul__(self, q: Qbits) -> Qbits:
        v = []
        for x in q.vector:
            v.append(x * self.vector[0])
        for x in q.vector:    
            v.append(x * self.vector[1])
        return Qbits(v)
    @multimethod
    def __rmul__(self, s: float) -> Qbit:
        return self.__mul__(s)
    @multimethod
    def __rmul__(self, s: int) -> Qbit:
        return self.__rmul__(float(s))

@dataclass
class Gate:
    matrix : List[List[float]]
    # 2x2 Matrix x  2 d column vector
    @multimethod
    def __mul__(self, q: Qbit) -> Qbit:
        return Qbit([self.matrix[0][0] * q.vector[0] + self.matrix[0][1] * q.vector[1], 
                     self.matrix[1][0] * q.vector[0] + self.matrix[1][1] * q.vector[1]])
    # n x n matrix * 1 x n column vector
    @multimethod
    def __mul__(self, q: Qbits) -> Qbits:
        v = [0.] * len(self.matrix)
        for r in range(len(self.matrix)):
            for c in range(len(self.matrix[0])):
                v[r] += self.matrix[r][c] * q.vector[c]
        return Qbits(v)

## Functions

In [36]:
def zero() -> Qbit:
    return Qbit([1,0])

def one() -> Qbit:
    return Qbit([0,1])

def qbits(t: str) -> Qbits | Qbit:
    #least significant bit to the right
    t = t[::-1]
    v = []
    for b in t:
        if b == '1':
            v.append(1)
        elif b == '0':
            v.append(0)
        else:
            raise Exception(f"Wrong qbit format '{b}'")
    if len(v) == 2:
        return Qbit(v)
    else:
        return Qbits(v)

def entangled_qbits(t: str) -> Qbits | Qbit:
    v = []
    #least significant bit to the right
    t = t[::-1]
    for b in t:
        if b == '1':
            v.append(one())
        elif b == '0':
            v.append(zero())
        else:
            raise Exception(f"Wrong qbit format '{b}'")
    if len(v) == 1:
        return v[0]
    else:
        return reduce(lambda x, y: x * y, v)

def ntobin(n: int, digits: int) -> [float]:
    v = []
    while n:
        v.append(float(n % 2))
        n = n // 2
    return v + [0] * (digits-len(v)) if digits > 0 else v

def qbit(d: int) -> Qbit:
    if d == 0:
        return zero()
    elif d == 1:
        return one()
    else:
        raise Exception(f"Invalid number '{d}'")

def zero() -> Qbit:
    return Qbit([1,0])

def one() -> Qbit:
    return Qbit([0,1])

def qbits(t: str) -> Qbits | Qbit:
    #least significant bit to the right
    t = t[::-1]
    v = []
    for b in t:
        if b == '1':
            v.append(1)
        elif b == '0':
            v.append(0)
        else:
            raise Exception(f"Wrong qbit format '{b}'")
    if len(v) == 2:
        return Qbit(v)
    else:
        return Qbits(v)

def disentangle(q: Qbits) -> [Qbits]:
    N = len(q.vector)
    print(q)
    QN = int(math.log2(N))
    print(N)
    v : List[Qbits] = []
    for i in range(N):
        v.append(q.vector[i] * Qbits(ntobin(i, QN)))
    return v

def bits_to_qbits(n, num) -> List[Qbit]:
    b = ntobin(n, num)
    return [qbit(int(i)) for i in b]

def bits_to_bits(n, num) -> List[Qbit]:
    b = ntobin(n, num)
    return [int(i) for i in b]

def measure_bits(q: Qbits) -> List[Tuple[float, List[int]]]:
    return [(e, bits_to_bits(i, int(math.log2(len(q))))) for (i, e) in enumerate(q.vector) if abs(e) > 1e-9]

def measure_prob_bits(q: Qbits) -> List[Tuple[float, List[int]]]:
    return [(e*e, bits_to_bits(i, int(math.log2(len(q))))) for (i, e) in enumerate(q.vector) if abs(e) > 1e-9]

def measure_qbits(q: Qbits) -> List[Tuple[float, List[Qbit]]]:
    return [(e, bits_to_qbits(i, int(math.log2(len(q))))) for (i, e) in enumerate(q.vector)]
    
    

## Gates

In [37]:
import math

def H(q: Qbit) -> Qbit:
    sq = 1/math.sqrt(2)
    return Gate([[sq,  sq],
                 [sq, -sq]]) * q
def X(q: Qbit) -> Qbit:
    return Gate([[0,1],[1,0]]) * q

def Y(q: Qbit) -> Qbit:
    return Gate([[0,-1j],[1j,0]]) * q

def Z(q: Qbit) -> Qbit:
    return Gate([[1, 0], [0,-1]])

def S(q: Qbit) -> Qbit:
    return Gate([[1,0], [0, 1j]])

def T(q: Qbit) -> Qbit:
    return Gate([[1,0], 
                 [0, math.cos(math.pi/4) + 1j * math.sin(math.pi/4)]])

def CZ(q: Qbits) -> Qbits:
    return Gate([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,1,0],
                 [0,0,0,-1]]) * q

def SWAP(q: Qbits) -> Qbits:
    return Gate([[1,0,0,0],
                 [0,0,1,0],
                 [0,1,0,0],
                 [0,0,0,1]]) * q

def CNOT(q: Qbits) -> Qbits:
    return Gate([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,0,1],
                 [0,0,1,0]]) * q

def Toffoli(q: Qbits) -> Qbits:
    return Gate([[1, 0, 0, 0, 0, 0, 0, 0],
                 [0, 1, 0, 0, 0, 0, 0, 0],
                 [0, 0, 1, 0, 0, 0, 0, 0],
                 [0, 0, 0, 1, 0, 0, 0, 0],
                 [0, 0, 0, 0, 1, 0, 0, 0],
                 [0, 0, 0, 0, 0, 1, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 1],
                 [0, 0, 0, 0, 0, 0, 1, 0]]) * q

CCNOT = Toffoli

## Circuits

In [38]:
c1 = H(zero())
c2 = one()
print(c1*c2)
print(disentangle(c1*c2))

Qbits(vector=[0.0, 0.0, 0.7071067811865475, 0.7071067811865475])
Qbits(vector=[0.0, 0.0, 0.7071067811865475, 0.7071067811865475])
4
[Qbits(vector=[0.0, 0.0]), Qbits(vector=[0.0, 0.0]), Qbits(vector=[0.0, 0.7071067811865475]), Qbits(vector=[0.7071067811865475, 0.7071067811865475])]


### Circuit representation.

Matrix:
 * num column = max number of stages
 * num rows = number of wires / qubits
 * element = gate with wire info e.g. H(wire1,wire2,...)
 * null element = id: output = input
```
[[H(0),  CNOT(0,1), H(0) ],
 [id(1), id(1),     id(1)]] 
```

### Simulation algorithm

1. disentangle --> retrieve all qbit possible states
2. if not last stage:
    1. compute all possible input states from output
    2. for each input state generate all possible output
    3. got to 1

In [39]:
t = H(one()) * one() * zero()
t

Qbits(vector=[0.0, -0.0, 0.7071067811865475, -0.7071067811865475, 0.0, -0.0, 0.0, -0.0])

In [40]:
measure_prob_bits(t)

[(0.4999999999999999, [0, 1, 0]), (0.4999999999999999, [1, 1, 0])]